In [1]:
# www.drsoli.com
# This is the full source code for Financial statements analysis post
# link to the post: https://drsoli.com/index.php/2023/09/18/10-k-financial-statements-analyzing-companies-reports-with-sec-edgar-api/

import requests
import json
from yfinance import download
from contextlib import suppress

In [2]:
CIK_list = [
  ["aapl",320193 ],
  ["msft",789019],
  ["xom",34088],
  ["nvda",1045810],
  ["tsla",1318605],
  ["pfe",78003],
  ["baba",1577552],
  ["csco",858877],
  ["adbe",796343],
  ["pypl",1633917],
  ["ibm",51143],
  ["nflx",1065280],
  ["ba",12927],
  ["wmt", 104169],
  ["jpm", 19617],
  ["orcl", 1341439],
  ["nke", 320187],
  ["amd", 2488],
  ["tgt", 27419],
  ["ge", 40545],
  ["googl", 1652044],
  ["amzn", 1018724],
  ["f", 37996]
 ]

In [3]:
def get_price(Ticker):
    try:
        data = download(tickers=Ticker, period='1d', interval='1h', progress=False)
        data = str(data['Close'][0])
        return data
    except Exception as err:
        print("getPrice error: " + str(err))
        return None

In [4]:
def get_raw_data(CIK):
  CIK = str(CIK)
  for i in range(10 - len(CIK)):
    CIK = "0" + CIK
  my_headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36" }
  data = requests.get("https://data.sec.gov/api/xbrl/companyfacts/CIK" + CIK + ".json", headers = my_headers)
  data = json.loads(data.text)
  return data

In [5]:
def calculate_ratios(data, ticker):
  current_ratio = "__"
  leverage_ratio = "__"
  PE_ratio = "__"
  inventory_turnover = "__"
  with suppress(KeyError):
    assets_current = (data["facts"]["us-gaap"]["AssetsCurrent"]["units"]["USD"][-1]["val"])
    liabilities_current = (data["facts"]["us-gaap"]["LiabilitiesCurrent"]["units"]["USD"][-1]["val"])
    current_ratio = assets_current / liabilities_current
  with suppress(KeyError):
    assets = (data["facts"]["us-gaap"]["Assets"]["units"]["USD"][-1]["val"])
    stockholders_equity = (data["facts"]["us-gaap"]["StockholdersEquity"]["units"]["USD"][-1]["val"])
    leverage_ratio = assets / stockholders_equity
  with suppress(KeyError):
    earnings_per_share_basic = (data["facts"]["us-gaap"]["EarningsPerShareBasic"]["units"]["USD/shares"][-1]["val"])
    share_price = get_price(ticker)
    PE_ratio = float(share_price) / earnings_per_share_basic
  try:
    inventory_net = (data["facts"]["us-gaap"]["InventoryNet"]["units"]["USD"][-1]["val"])
    try:
      cost_of_goods_sold = (data["facts"]["us-gaap"]["CostOfGoodsSold"]["units"]["USD"][-1]["val"])
    except:
      try:
        cost_of_goods_sold = (data["facts"]["us-gaap"]["CostOfGoodsAndServicesSold"]["units"]["USD"][-1]["val"])
      except:
        try:
          cost_of_goods_sold = (data["facts"]["us-gaap"]["CostOfRevenue"]["units"]["USD"][-1]["val"]) - (data["facts"]["us-gaap"]["MarketingAndAdvertisingExpense"]["units"]["USD"][-1]["val"])
        except:
          cost_of_goods_sold = (data["facts"]["us-gaap"]["CostOfRevenue"]["units"]["USD"][-1]["val"]) - (data["facts"]["us-gaap"]["AdvertisingExpense"]["units"]["USD"][-1]["val"])
    inventory_turnover = cost_of_goods_sold / inventory_net
  except:
    pass
  ratios = [current_ratio, leverage_ratio, PE_ratio, inventory_turnover]
  ratios = [ratio if isinstance(ratio, str) else str(round(ratio,3)) for ratio in ratios]
  return ratios

In [6]:
for [Ticker, CIK] in CIK_list:
  data = get_raw_data(CIK)
  print(Ticker, "\t\t", data["facts"]["us-gaap"]["Assets"]["units"]["USD"][-1]["val"])

aapl 		 337411000000
msft 		 484275000000
xom 		 377918000000
nvda 		 77072000000
tsla 		 109226000000
pfe 		 221095000000
baba 		 244426000000
csco 		 122998000000
adbe 		 28751000000
pypl 		 83347000000
ibm 		 137169000000
nflx 		 48827721000
ba 		 134484000000
wmt 		 254054000000
jpm 		 4090727000000
orcl 		 137082000000
nke 		 37356000000
amd 		 67895000000
tgt 		 55117000000
ge 		 163942000000
googl 		 407350000000
amzn 		 530969000000
f 		 274341000000


In [7]:
print("%12s" % "Ticker", "%15s" % "Current", "%15s" %  "Leverage", "%15s" % "P/E", "%25s" % "Inventory Turnover")
print("     ___________________________________________________________________________________\n")
for [Ticker, CIK] in CIK_list:
  data = get_raw_data(CIK)
  ratios = calculate_ratios(data, Ticker)
  print("%12s" % Ticker, "%15s" % ratios[0], "%15s" % ratios[1], "%15s" % ratios[2], "%25s" % ratios[3])

      Ticker         Current        Leverage             P/E        Inventory Turnover
     ___________________________________________________________________________________

        aapl           1.037           4.548         139.471                      7.78
        msft           1.242           1.913         149.603                     2.627
         xom           1.382           1.841          52.777                        __
        nvda           3.529           1.568           21.96                     1.071
        tsla           1.716           1.697         484.351                     0.158
         pfe           1.047           2.396          49.964                     0.235
        baba           1.786           1.789         134.273                        __
        csco           0.895           2.687          95.803                     1.163
        adbe           1.049            1.86         383.015                        __
        pypl           1.278           4